In [6]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler goatools gseapy scperturb chembl_webresource_client biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

In [7]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns
import scvelo as scv
scv.settings.verbosity=1

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../../')
from utils import *

# scperturb package
sys.path.insert(1, '../../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../../figures/')

In [8]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')
DATADIR = Path('/home/peidli/data/scPerturb/')

In [29]:
from scipy.io import mmread
from scipy.sparse import csr_matrix

In [ ]:
X = mmread(TEMPDIR / 'SunshineHein2023/matrix.mtx')
obs = pd.read_csv(TEMPDIR / 'SunshineHein2023/barcodes.tsv.gz', index_col=0, sep='\t', names=['cell_barcode'])
var = pd.read_csv(TEMPDIR / 'SunshineHein2023/features.tsv.gz', index_col=1, sep='\t', names=['ensembl_id', 'gene_symbol', 'feature_type'])
ids = pd.read_csv(TEMPDIR / 'SunshineHein2023/cell_identities.csv', index_col=0)

In [31]:
adata = sc.AnnData(csr_matrix(X).T, pd.concat([obs, ids], axis=1), var)
adata.var.drop('feature_type', axis=1, inplace=True)  # trivial
adata.var_names_make_unique()

In [46]:
# move non-gene features to obsm
group1 = adata.var.index.str.startswith('SCV_')
non_genes = list(adata.var_names[group1])
adata.obsm['SCOV_expression'] = pd.DataFrame(adata[:, non_genes].X.A, index=adata.obs_names, columns=non_genes)
adata = adata[:, ~group1].copy()

group2 = adata.var.index.str.startswith('lenti_')
non_genes = list(adata.var_names[group2])
adata.obsm['lentivirus_capture'] = pd.DataFrame(adata[:, non_genes].X.A, index=adata.obs_names, columns=non_genes)
adata = adata[:, ~adata.var.index.str.startswith('lenti_')].copy()

In [47]:
adata.obs

,guide_identity,number_of_guides,read_count,UMI_count,coverage,gem_group,good_coverage,number_of_cells,matched_library_element,match_type
cell_barcode,,,,,,,,,,
AAACCTGAGAAACCTA-1,BCKDK_+_31119819.23-P1P2_CR3-cs1;EIF2AK3_-_889...,2,812;526,583;384,1.3927958833619212;1.3697916666666667,1,True,1.0,NaN,scrambled_pair
AAACCTGAGAATTCCC-1,COG6_-_40229866.23-P1P2_CR3-cs1;SIGMAR1_+_3463...,3,29;290;116,20;210;77,1.45;1.380952380952381;1.5064935064935066,1,True,2.0,NaN,multiplet
AAACCTGAGACAAAGG-1,NaN,0,0,0,0,1,False,NaN,NaN,no_match
AAACCTGAGCCGATTT-1,EIF2AK2_+_37384079.23-P1P2_CR1-cs1;STK16_+_220...,3,483;541;3548,362;385;2562,1.3342541436464088;1.405194805194805;1.3848555...,1,True,2.0,NaN,multiplet
AAACCTGAGCGAGAAA-1,AP3B1_-_77590427.23-P1P2_CR1-cs1;CCZ1_-_593859...,4,216;805;156;54,164;583;101;41,1.3170731707317074;1.3807890222984562;1.544554...,1,True,2.0,NaN,multiplet
...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCATGGTCA-8,NaN,0,0,0,0,8,False,NaN,NaN,no_match
TTTGTCATCCCAAGAT-8,C19orf66_-_10197533.23-P1P2_CR1-cs1,1,74,55,1.3454545454545455,8,True,1.0,CRISPRi_216_C19orf66,exact_match
TTTGTCATCGGTGTTA-8,TMEM106B_+_12250990.23-P1P2_CR1-cs1,1,330,238,1.3865546218487395,8,True,1.0,CRISPRi_234_TMEM106B,exact_match


In [49]:
# what do the numbers mean? I guess the +/- means strand???
list(adata.obs.guide_identity.unique())

['BCKDK_+_31119819.23-P1P2_CR3-cs1;EIF2AK3_-_88926935.23-P1P2_CR1-cs1',
 'COG6_-_40229866.23-P1P2_CR3-cs1;SIGMAR1_+_34637739.23-P1P2_CR3-cs1;TMEM97_-_26646271.23-P1P2_CR1-cs1',
 nan,
 'EIF2AK2_+_37384079.23-P1P2_CR1-cs1;STK16_+_220110240.23-P1P2_CR1-cs1;STK16_+_220110286.23-P1P2_CR3-cs1',
 'AP3B1_-_77590427.23-P1P2_CR1-cs1;CCZ1_-_5938597.23-P1P2_CR3-cs1;EIF3F_-_8009270.23-P1P2_CR3-cs1;RBX1_+_41347427.23-P1P2_CR1-cs1',
 'TRMT1_-_13227169.23-P1P2_CR3-cs1',
 'STAT2_+_56753865.23-P1P2_CR1-cs1;STAT1_-_191878853.23-P1P2_CR3-cs1',
 'IFITM2_+_308180.23-P1P2_CR3-cs1;IFITM2_+_308197.23-P1P2_CR1-cs1',
 'TOM1_+_35695890.23-P1P2_CR1-cs1',
 'BST2_+_17516392.23-P1P2_CR3-cs1;IFIH1_-_163175124.23-P1P2_CR1-cs1',
 'DDX1_-_15732017.23-P1P2_CR3-cs1;ACE2_-_15619013.23-P1P2_CR3-cs1;BTF3_-_72794314.23-P1P2_CR3-cs1;CHMP4B_-_32399280.23-P1P2_CR3-cs1;TMPRSS2_-_42879996.23-P1P2_CR3-cs1;RELB_+_45504764.23-P1P2_CR3-cs1;BRD4_-_15391126.23-ENST00000263377.2_ENST00000371835.4_CR1-cs1;CHMP4B_-_32399446.23-P1P2_CR1-cs1;

In [ ]:
# harmonize metadata
adata.obs = adata.obs.rename({'guide_identity': 'perturbation', 
                              'number_of_guides': 'nperts', 
                              '': ''
                             }, axis=1)
adata.obs = adata.obs.drop(['gem_group'], axis=1)  # irrelevant for general audience



adata.obs.perturbation = adata.obs.perturbation.replace({'Non-Targeting': 'control'})  # Non-targeting = control
adata = adata[adata.obs.perturbation!='nan'].copy()  # barcode undetermined
adata.obs = adata.obs[['perturbation', 'nguides_detected', 'guides_call', 'guidewise_counts', 'guide_ncounts', 'guide_id', 'ChromHMM_chromatin_state', 'ncounts']] # reorder
adata.obs = adata.obs[['perturbation', 'nguides_detected', 'guides_call', 'guidewise_counts', 'guide_ncounts', 'guide_id', 'ChromHMM_chromatin_state', 'ncounts']] # reorder
adata.obs['perturbation_type'] = 'CRISPRa'
adata.obs['disease'] = "healthy"
adata.obs['cancer'] = False
adata.obs['tissue_type']="cell_line"
adata.obs["cell_line"] = "hPSCs"
adata.obs["celltype"] = 'stem cells'
adata.obs['organism'] = 'human'
adata.obs.perturbation = [x.replace('-', '_') for x in adata.obs.perturbation]  # convention for double perturbations
adata.obs['nperts'] = [p.count('_')+1-p.count('control') if type(p)==str else 0 for p in adata.obs.perturbation]